In [ ]:
!pip install huggingface_hub
!pip install datasets
!pip install ipywidgets
!git clone https://github.com/Chris1nexus/community-events.git
%cd community-events
!pip install .
%cd huggan/pytorch/cyclegan
!pip install wandb
!wandb login
!accelerate config

In [4]:
from sklearn.model_selection import ParameterGrid
from subprocess import call
import os



hyperparams_dcgan = {
    
    'lr':[0.0002], 
    'latent_dim':[100],
        'generator_hidden_size':[64],
    'discriminator_hidden_size':[64],
    
    
        'image_size':[64], 
        'num_channels':[3], 
    'dataset':['Chris1/cryptopunks'],  
'num_workers':[8], 
     'batch_size':[8],
    
    'num_epochs':[200],  
        'beta1':[0.5], 
 #'fp16':[False],
    'mixed_precision':['no'], 
    'output_dir':['./output'],
    #'cpu':[False], 
    'wandb':[True],
    'logging_steps':[50],
    'push_to_hub':[True], 
    'model_name':['test'],
     'organization_name':['Chris1'], 
    
    
                     }


In [5]:
import copy
import pandas as pd
EXPERIMENTS_DATA_CSV = 'experiments.csv'

if not os.path.exists(EXPERIMENTS_DATA_CSV):
    experiments_df = pd.DataFrame()
else:
    experiments_df = pd.read_csv(EXPERIMENTS_DATA_CSV)



for idx, grid in enumerate(ParameterGrid(hyperparams_dcgan)):

        copy_grid = copy.deepcopy(grid)
        if 'dataset_name' in copy_grid:
            del copy_grid['dataset_name']
        if 'dataset' in copy_grid:
            del copy_grid['dataset']
        if 'output_dir' in copy_grid:
            del copy_grid['output_dir']
            
        experiment_name =  '--'.join([f'{k}__{v}'  for k,v in copy_grid.items() ])
        call_params = ['accelerate', 'launch',
                       '--config_file','~/.cache/huggingface/accelerate/default_config.yaml',
                       'train.py']
        
        
        experiment_id = len(experiments_df)
        grid['output_dir'] = experiment_name
        
        experiments_df.append(grid,ignore_index=True)
        grid['output_dir'] = f'experiments/{experiment_id}'
        for k,v in grid.items():
            
             
            call_params.append(f'--{k}' )
            if k not in  ['cpu', 'wandb','fp16','push_to_hub']:
                call_params.append(f'{v}' )
        call(call_params)

Namespace(batch_size=8, beta1=0.5, cpu=True, dataset='Chris1/cryptopunks', discriminator_hidden_size=64, fp16=True, generator_hidden_size=64, image_size=64, latent_dim=100, logging_steps=50, lr=0.0002, mixed_precision='no', model_name='test', num_channels=3, num_epochs=200, num_workers=8, organization_name='Chris1', output_dir=PosixPath('experiments/0'), push_to_hub=True, wandb=True)
Namespace(batch_size=8, beta1=0.5, cpu=True, dataset='Chris1/cryptopunks', discriminator_hidden_size=64, fp16=True, generator_hidden_size=64, image_size=64, latent_dim=100, logging_steps=50, lr=0.0002, mixed_precision='no', model_name='test', num_channels=3, num_epochs=200, num_workers=8, organization_name='Chris1', output_dir=PosixPath('experiments/0'), push_to_hub=True, wandb=True)


wandb: Currently logged in as: chris1nexus (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.14 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.12.12
wandb: Run data is saved locally in /home/xrh1/experiments/huggan/community-events/huggan/pytorch/dcgan/wandb/run-20220409_145545-6yvnkbz4
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run glorious-haze-2
wandb: ⭐️ View project at https://wandb.ai/chris1nexus/0
wandb: 🚀 View run at https://wandb.ai/chris1nexus/0/runs/6yvnkbz4
Downloading: 100%|██████████| 642/642 [00:00<00:00, 1.12MB/s]


100%|██████████| 1/1 [00:00<00:00, 891.46it/s]
100% 1/1 [00:00<00:00, 1131.46it/s]
INFO:__main__:***** Running training *****
INFO:__main__:  Num Epochs = 200


Dataset parquet downloaded and prepared to /home/xrh1/.cache/huggingface/datasets/parquet/Chris1--cryptopunks-aa3e6d933fa2881e/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


INFO:__main__:| epoch: 0.000e+00| discriminator_loss: 2.064e-06| generator_loss: 3.503e+01| D_x: 1.000e+00| D_G_z1: 3.622e-15| D_G_z2: 3.632e-15
INFO:__main__:| epoch: 0.000e+00| discriminator_loss: 1.336e-05| generator_loss: 3.711e+01| D_x: 1.000e+00| D_G_z1: 1.239e-15| D_G_z2: 1.252e-15
INFO:__main__:| epoch: 0.000e+00| discriminator_loss: 1.375e-04| generator_loss: 3.657e+01| D_x: 9.997e-01| D_G_z1: 2.920e-16| D_G_z2: 2.931e-16
INFO:__main__:| epoch: 0.000e+00| discriminator_loss: 4.167e-05| generator_loss: 3.411e+01| D_x: 1.000e+00| D_G_z1: 7.941e-13| D_G_z2: 7.991e-13
INFO:__main__:| epoch: 0.000e+00| discriminator_loss: 8.373e-01| generator_loss: 1.239e+01| D_x: 9.967e-01| D_G_z1: 3.481e-01| D_G_z2: 4.254e-05
INFO:__main__:| epoch: 0.000e+00| discriminator_loss: 7.056e-01| generator_loss: 9.544e+00| D_x: 9.888e-01| D_G_z1: 3.694e-01| D_G_z2: 2.263e-04
INFO:__main__:| epoch: 0.000e+00| discriminator_loss: 1.969e-02| generator_loss: 9.495e+00| D_x: 9.809e-01| D_G_z1: 6.074e-04| D_G

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
Killing subprocess 1833733
Killing subprocess 1833734
Main process received SIGINT, exiting


Traceback (most recent call last):
  File "/home/xrh1/experiments/huggan/hugenv/bin/accelerate", line 8, in <module>
    sys.exit(main())
  File "/home/xrh1/experiments/huggan/hugenv/lib/python3.8/site-packages/accelerate/commands/accelerate_cli.py", line 43, in main
    args.func(args)
  File "/home/xrh1/experiments/huggan/hugenv/lib/python3.8/site-packages/accelerate/commands/launch.py", line 468, in launch_command
    multi_gpu_launcher(args)
  File "/home/xrh1/experiments/huggan/hugenv/lib/python3.8/site-packages/accelerate/commands/launch.py", line 226, in multi_gpu_launcher
    process.wait()
  File "/usr/lib/python3.8/subprocess.py", line 1083, in wait
    return self._wait(timeout=timeout)
  File "/usr/lib/python3.8/subprocess.py", line 1806, in _wait
    (pid, sts) = self._try_wait(0)
  File "/usr/lib/python3.8/subprocess.py", line 1764, in _try_wait
    (pid, sts) = os.waitpid(self.pid, wait_flags)
KeyboardInterrupt


KeyboardInterrupt: 